In [3]:
import ee
import os
import geojson
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as disp
from geemap import png_to_gif
from geemap.cartoee import get_map, add_gridlines, add_scale_bar_lite, add_north_arrow
import requests
ee.Initialize()
%matplotlib inline

In [18]:
os.listdir("Locations/")

['algeciras.geojson',
 'ferrol.geojson',
 'gibraltar.geojson',
 'gibraltar_peq.geojson',
 'map.geojson',
 'shangai.geojson',
 'south_sicilia.geojson']

In [39]:
filename = "gibraltar.geojson"
with open("Locations\\" + filename) as f:
    gj = geojson.load(f)

coords = gj['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)


start_date = "2021-12-01" # 
end_date = "2021-12-31" #

ffa_db = (ee.ImageCollection('COPERNICUS/S1_GRD')
                       .filterBounds(aoi)
                       .filterDate(ee.Date(start_date), ee.Date(end_date))
                       .sort('system:time_start'))

#import time
#acq_times = ffa_db.aggregate_array('system:time_start').getInfo()
#[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

im_list = ffa_db.toList(ffa_db.size())
index = 9

mode = "VV"

if mode == "VV":
    min_vision = -30
    max_vision = 0
elif mode == "VH":
    min_vision = -20
    max_vision = 0

url = ee.Image(im_list.get(index)).select(mode).clip(aoi).getThumbURL({'min': min_vision, 'max': max_vision})
disp.Image(url=url, width=1200)

In [9]:
ee.Image(im_list.get(index)).select('VH').clip(aoi).getInfo()

{'type': 'Image',
 'bands': [{'id': 'VV',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [5797, 4566],
   'origin': [4770, 16914],
   'crs': 'EPSG:32633',
   'crs_transform': [10, 0, 408097.03989331954, 0, -10, 4222576.922648753]}],
 'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20211206T165627_20211206T165652_040891_04DAFD_DFA1',
 'version': 1643908686275680.0,
 'properties': {'GRD_Post_Processing_start': 1638812202459,
  'sliceNumber': 5,
  'GRD_Post_Processing_facility_name': 'Copernicus S1 Core Ground Segment - TLS',
  'resolution': 'H',
  'SLC_Processing_facility_name': 'Copernicus S1 Core Ground Segment - TLS',
  'system:footprint': {'type': 'Polygon',
   'coordinates': [[[14.50882, 36.242536910357295],
     [15.110854955330593, 36.33342693404028],
     [15.152893000000004, 36.33965239622648],
     [15.152893000000004, 36.625448],
     [14.50882, 36.625448],
     [14.50882, 36.242536910357295]]]},
  'familyName': 'SENTINEL-1',
  'segmentStartTime': 163

In [40]:

img_data = requests.get(url).content
with open('Images/prueba2.png', 'wb') as handler:
    handler.write(img_data)